# Lesson 3: Reflection and Blogpost Writing

## Setup

In [3]:
# !pip install autogen openai

In [4]:
# reset all those keys
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [6]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


# Goal:

Use nested chat conversation pattern to realize a sophisticated reflect process; build a system where a set of reviewer agents is nested within a querying agent as in the monologue to reflect on a blog post written by a writer agent.

## Create a writer agent

In [9]:
# !pip install dask

In [10]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

# reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
# print(reply)

In [11]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [13]:
print(reply)
# the reply's contents come from what we have defined in "Tasks"

Title: Unleashing the Power of Deep Learning with DeepLearning.AI

Are you ready to dive into the world of deep learning and artificial intelligence? Look no further than DeepLearning.AI! With comprehensive courses taught by industry experts like Andrew Ng, this platform is your gateway to mastering deep learning algorithms and applications. Whether you're a beginner or looking to advance your skills, DeepLearning.AI offers a range of courses to suit your needs. Join the millions of learners worldwide who are upskilling with DeepLearning.AI and unlock exciting opportunities in the ever-evolving field of AI. Start your deep learning journey today!


## Adding reflection

Create a critic agent to reflect on the work of the writer agent.


- use another agent called critic to reflect on the first agent writer.

such as:
- write a concise but engaging blopost about deeplearning.ai
  - answer: deeplearning.ai is awesome.....
- critic: overall, the SEO reviewer suggests ... the legal reviewer suggests.... in conclusion, it is essential to ....
  - answer: explore the transformative world of AI education with deeplearning.ai! let by ai expertAndrew ng.....

  hopefully, the answers get better.

In [14]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the World of DeepLearning.AI"

Are you ready to dive into the innovative world of artificial intelligence and deep learning? Look no further than DeepLearning.AI! From beginner to advanced courses, DeepLearning.AI offers a plethora of learning opportunities designed to help you master the complexities of AI technologies. With expert-led courses and hands-on projects, you can sharpen your skills and stay ahead in this rapidly evolving field. Whether you're a coding enthusiast or a tech professional, DeepLearning.AI has something for everyone. Get ready to unlock the secrets of AI and transform your career with DeepLearning.AI today!

--------------------------------------------------------------------------------


## Nested chat

In [16]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [17]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [18]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [22]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

# aggregate all the reviews from the reviewers.
# the next step is to define the task to be registered.
# here with further use the chat conversation pattern, leaned in the last lesson to construct the chat and the critic.

## Orchestrate the nested chats to solve the task

In [23]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


# 4 different chats and different review

In [24]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [25]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the World of DeepLearning.AI"

Are you ready to dive into the innovative world of artificial intelligence and deep learning? Look no further than DeepLearning.AI! From beginner to advanced courses, DeepLearning.AI offers a plethora of learning opportunities designed to help you master the complexities of AI technologies. With expert-led courses and hands-on projects, you can sharpen your skills and stay ahead in this rapidly evolving field. Whether you're a coding enthusiast or a tech professional, DeepLearning.AI has something for everyone. Get ready to unlock the secrets of AI and transform your career with DeepLearning.AI today!

--------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:1264: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

As a legal reviewer, I would suggest the following:

- Ensure that any claims made about the benefits or outcomes of using DeepLearning.AI are backed up by accurate and substantiated evidence to avoid any potential false advertising issues.

- Verify that the statements regarding the expertise of the instructors at DeepLearning.AI are truthful and verifiable to prevent any claims of misleading information.

- Consider including a disclaimer about the intended educational purposes of the courses offered by DeepLearning.AI to mitigate any possible confusion or misinterpretation by users.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Ethics Reviewer):

Review the following content. 
            

 Title: "Exploring the World of D

## Get the summary

In [26]:
print(res.summary)

Title: "Unveiling the Marvels of DeepLearning.AI"

Embark on an exhilarating journey into the realm of artificial intelligence with DeepLearning.AI! Offering a variety of courses suitable for all skill levels, this platform is a beacon of knowledge in the fast-evolving field of AI technology. Whether you're a novice or an experienced professional, DeepLearning.AI provides the tools and guidance to help you navigate through the intricacies of deep learning. Dive deep into hands-on projects and expert-led sessions to elevate your understanding and expertise. Join the ranks of AI enthusiasts and tech aficionados who are shaping the future with DeepLearning.AI today!


# put it all together

In [29]:
# define task
task = '''
        Write a concise but engaging blogpost about
       the jannik sinner, especially include his recent performance in
       China open and Shanghai master.
       '''
# create a writer agent
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)
# define reply
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
reply

"Title: Unleashing Greatness: Jannik Sinner's Spectacular Run in the China Open and Shanghai Masters\n\nYoung tennis sensation Jannik Sinner has been making waves in the tennis world with his outstanding performance in the China Open and Shanghai Masters. The 20-year-old Italian prodigy showcased his exceptional talent, dedication, and unwavering determination on the court, leaving fans and critics in awe.\n\nIn the China Open, Sinner's powerful serves and precise shots overwhelmed his opponents, leading him to secure impressive victories and advance to the later stages of the tournament. His strategic gameplay and remarkable composure under pressure set him apart, proving that he has the makings of a future tennis legend.\n\nNot stopping there, Sinner continued his winning streak at the Shanghai Masters, displaying a level of skill and finesse that belied his young age. With each match, he demonstrated his ability to adapt to different playing styles and overcome challenges with grace

In [30]:
# adding reflections:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       the jannik sinner, especially include his recent performance in 
       China open and Shanghai master. 
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: A Rising Star: Jannik Sinner's Spectacular Performance in China Open and Shanghai Masters

Jannik Sinner, the Italian tennis sensation, has been making waves in the tennis world with his remarkable performances in the recent China Open and Shanghai Masters. The 19-year-old prodigy has been leaving fans and critics alike in awe with his incredible skills and unwavering determination on the court.

In the China Open, Sinner showed his prowess by defeating some of the top-ranked players, showcasing his powerful serves and impressive court coverage. His strategic gameplay and fearless attitude set him apart, making him a force to be reckoned with in the tennis world.

Not stopping there, Sin

In [36]:
# nested chat:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)
# self defined reviewer: style reviewer
style_reviewer = autogen.AssistantAgent(
    name="Style Reviewer",
    llm_config=llm_config,
    system_message="You are a style reviewer, known for "
        "your ability to ensure that content is written in a fun and chill way and with a sense of humor."
        "and free from adding any stressor the reader."
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [37]:
# Orchestrate the nested chats to solve the task
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {
     "recipient": style_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


In [38]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [39]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       the jannik sinner, especially include his recent performance in 
       China open and Shanghai master. 
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: A Rising Star: Jannik Sinner's Spectacular Performance in China Open and Shanghai Masters

Jannik Sinner, the Italian tennis sensation, has been making waves in the tennis world with his remarkable performances in the recent China Open and Shanghai Masters. The 19-year-old prodigy has been leaving fans and critics alike in awe with his incredible skills and unwavering determination on the court.

In the China Open, Sinner showed his prowess by defeating some of the top-ranked players, showcasing his powerful serves and impressive court coverage. His strategic gameplay and fearless attitude set him apart, making him a force to be reckoned with in the tennis world.

Not stopping there, Sin

In [40]:
print(res.summary)

Title: Rising to the Challenge: Jannik Sinner's Triumphs in China Open and Shanghai Masters

Jannik Sinner, the young tennis sensation from Italy, has been setting the tennis world abuzz with his outstanding performances in the recent China Open and Shanghai Masters. At just 19 years old, Sinner has showcased not only his powerful serves and court coverage but also his unwavering determination that sets him apart from his competitors.

In the China Open, Sinner displayed his prowess by defeating top-ranked players, demonstrating not only his strategic gameplay but also his fearlessness on the court. His ability to stay cool under pressure and adapt to different playing styles has earned him admiration from fans and experts alike.

Continuing his winning streak in the Shanghai Masters, Sinner proved his versatility on various court surfaces. His tenacity and fighting spirit were on full display, securing him well-deserved victories and accolades. However, it was not just his victories t